In [1]:
from sympy.parsing.sympy_parser import parse_expr

In [2]:
parse_expr("0.3x + 3y", evaluate=True, local_dict={"x": 10, "y": 3, "z": 5}, transformations="all")

12

In [3]:
# from sympy import Abs
from sympy import *
from sympy.abc import x

parse_expr("max(1, 2, 3)", local_dict={"x": -10}, evaluate=True)

3

In [4]:
from numericals import Numerical

numerical = Numerical()

numerical.summarize_historical_data("AAPL")

[*********************100%***********************]  1 of 1 completed


{'period': '100d',
 'interval': '1d',
 'time_zone': None,
 'average_open': 150.29220054626464,
 'max_open': 173.0500030517578,
 'min_open': 126.01000213623047,
 'stdev_open': 12.932928708865637}

In [5]:
from expression import Expression

expr = Expression(str_expression="3x^2 + average(AAPL_return, 10d, 1m) * average(what, 5d, 1s)", variable_dict={"x": 10})
expr.evaluate("3x^2 + _average(AAPL_adj_close, 10d, 1d) * _average(MSFT_volume, 5d, 1d)")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


3561732671.7178106

In [103]:
import re 

str_ = "what(andif, always, thes(whatever_what,3d ,1d,offf), the_fuck)"

In [104]:
re.findall(r"\w+\(\w+\s*?,\s*?\w+", str_)

['what(andif, always', 'thes(whatever_what,3d']

In [6]:
"_".join("what_is_this".split("_")[1:])

'is_this'

In [7]:
for f in funs:
    print(funs[f][0])

NameError: name 'funs' is not defined

In [ ]:
for f in funs:
    x = expr.evaluate_function(function_=f, expression=funs[f][0])
    # print(funs[f])
    print(x)

# x = funs['average'][0]
# x.replace("average(", "").replace(")", "").replace(" ", "").split(",")


[*********************100%***********************]  1 of 1 completed
Date
2023-04-24     41949600
2023-04-25     48714100
2023-04-26     45498800
2023-04-27     64902300
2023-04-28     55209200
2023-05-01     52472900
2023-05-02     48425700
2023-05-03     65136000
2023-05-04     81235400
2023-05-05    113316400
Name: volume, dtype: int64
61686040.0


In [ ]:
from numericals import Numerical

what = Numerical()
what.get_historical_data("AAPL")

[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adj_close,volume,return
Date,,,,,,,
2022-12-12,142.699997,144.500000,141.059998,144.490005,144.269730,70462700,NaN
2022-12-13,149.500000,149.970001,144.240005,145.470001,145.248230,93886200,0.006782
2022-12-14,145.350006,146.660004,141.160004,143.210007,142.991684,82291200,-0.015536
2022-12-15,141.110001,141.800003,136.029999,136.500000,136.291901,98931900,-0.046854
2022-12-16,136.690002,137.649994,133.729996,134.509995,134.304932,160156900,-0.014579
...,...,...,...,...,...,...,...
2023-05-01,169.279999,170.449997,168.639999,169.589996,169.589996,52472900,-0.000530
2023-05-02,170.089996,170.350006,167.539993,168.539993,168.539993,48425700,-0.006191
2023-05-03,169.500000,170.919998,167.160004,167.449997,167.449997,65136000,-0.006467
